In [ ]:
# Import necessary libraries: Flask for web app, render_template for rendering HTML templates,
# request to handle requests, and requests library for making HTTP requests.
from flask import Flask, render_template, request
import requests
import json

# Initialize the Flask app
app = Flask(__name__)

# Define the route for the homepage, supporting both GET and POST methods.
@app.route('/', methods=['GET', 'POST'])
def index():
    # If the request method is POST, handle the form submission.
    if request.method == 'POST':
        return handle_request()
    # If the request method is GET, render the homepage HTML template.
    return render_template('index.html')

# Define a function to fetch grid point data from the National Weather Service API.
def fetch_gridpoint_data(latitude, longitude):
    """Fetch WFO, gridX, and gridY using latitude and longitude."""
    base_url = 'https://api.weather.gov/'
    url = f"{base_url}points/{latitude},{longitude}"
    response = requests.get(url)  # Make the API request
    data = response.json()  # Parse the response as JSON

    # Extract the required data from the response
    wfo = data['properties']['gridId']
    gridX = data['properties']['gridX']
    gridY = data['properties']['gridY']

    # Return the Weather Forecast Office (WFO), gridX, and gridY values
    return wfo, gridX, gridY

# Define a function to handle form requests and make API calls based on the form data.
def handle_request():
    api_type = request.form.get('apiType')  # Get the type of API request from the form
    base_url = 'https://api.weather.gov/'

    # Handle requests for station data
    if api_type == 'stations':
        # Extract necessary parameters from the form
        station_id = request.form.get('id')
        state = request.form.get('state')
        limit = request.form.get('limit', type=int)  # Default type for limit is int
        cursor = request.form.get('cursor')  # For pagination, if necessary

        # Construct the URL for the stations API endpoint
        url = f"{base_url}stations?state={state}&limit={limit}"
        if station_id:
            url += f"&stationIdentifier={station_id}"

    # Handle requests for point data
    elif api_type == 'point':
        latitude = request.form.get('latitude')
        longitude = request.form.get('longitude')
        # Construct the URL for the points API endpoint
        url = f"{base_url}points/{latitude},{longitude}"

    # Handle requests for gridpoints data
    elif api_type == 'gridpoints':
        latitude = request.form.get('latitude')
        longitude = request.form.get('longitude')

        # If latitude and longitude are provided, fetch gridpoint data
        if latitude and longitude:
            wfo, gridX, gridY = fetch_gridpoint_data(latitude, longitude)
        else:
            # Otherwise, use the provided WFO, gridX, and gridY values
            wfo = request.form.get('wfo')
            gridX = request.form.get('gridX')
            gridY = request.form.get('gridY')

        # Construct the URL for the gridpoints API endpoint
        url = f"{base_url}gridpoints/{wfo}/{gridX},{gridY}"

    # Make the API request and return the JSON response
    response = requests.get(url)
    return json.dumps(response.json(), indent=4)

# Run the app only if this script is executed directly
if __name__ == '__main__':
    app.run(debug=False)  # Turn off debug mode for production

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Feb/2024 12:21:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2024 12:21:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/Feb/2024 12:21:35] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2024 12:21:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2024 12:21:55] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2024 12:21:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2024 12:22:05] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2024 12:22:21] "GET / HTTP/1.1" 200 -
